In [ ]:
!pip install contextily geopandas folium mapclassify tensorflow tqdm ipdb leafmap rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import leafmap
import rasterio
import os
import time
import requests
import json
import math
import matplotlib.pyplot as plt
import contextily as cx
import seaborn as sns
from tensorflow.keras.models import load_model
from pyproj import Transformer
from shapely.geometry import Point
from PIL import Image
from tqdm import tqdm
from shapely import wkt
from rasterio.mask import mask
from rasterio.enums import Compression
from shapely.geometry import mapping, box
from matplotlib import pyplot as plt
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import output
output.enable_custom_widget_manager()

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from PIL import Image
import rasterio
from rasterio.mask import mask
import leafmap

def convert_tif_to_png(input_tif, output_png, max_size_mb=15):
    with rasterio.open(input_tif) as src:
        image_array = src.read()
        image_array = np.moveaxis(image_array, 0, -1)

        if image_array.shape[-1] == 1:
            image_array = np.repeat(image_array, 3, axis=-1)

        alpha_channel = np.where((image_array[..., 0] == 0) &
                                 (image_array[..., 1] == 0) &
                                 (image_array[..., 2] == 0), 0, 255).astype(np.uint8)

        img = Image.fromarray(np.dstack((image_array, alpha_channel)), mode="RGBA")
        os.makedirs(os.path.dirname(output_png), exist_ok=True)
        img.save(output_png)

        # Check file size and resize proportionally if necessary
        max_size_bytes = max_size_mb * 1024 * 1024
        file_size = os.path.getsize(output_png)
        if file_size > max_size_bytes:
            scale_factor = (max_size_bytes / file_size) ** 0.5  # Compute scale factor
            width, height = img.size
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)
            img = img.resize((new_width, new_height), Image.LANCZOS)
            img.save(output_png)


def census_satellite_image(gdf_census, census_id, output):
    polygon = gdf_census[gdf_census['SEZ21_ID'] == census_id].iloc[0].geometry
    min_lon, min_lat, max_lon, max_lat = polygon.bounds
    bbox = [min_lon, min_lat, max_lon, max_lat]

    filename = f'{census_id}'
    image = f'{filename}.tif'
    cropped_image = f'cropped_{filename}.tif'
    label = gdf_census[gdf_census['SEZ21_ID'] == census_id]['COD_TIPO_S'].iloc[0]
    png = f'{output}/{label}/{filename}.png'

    leafmap.map_tiles_to_geotiff(
        output=image,
        bbox=bbox,
        zoom=19,
        source="Satellite",
        overwrite=True
    )

    with rasterio.open(image) as src:
        raster_crs = src.crs

    if gdf_census.crs != raster_crs:
        gdf_census = gdf_census.to_crs(raster_crs)

    transformed_polygon = gdf_census[gdf_census['SEZ21_ID'] == census_id].iloc[0].geometry

    with rasterio.open(image) as src:
        out_image, out_transform = mask(src, [transformed_polygon], crop=True)
        out_meta = src.meta.copy()

        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

    with rasterio.open(cropped_image, "w", **out_meta) as dest:
        dest.write(out_image)

    convert_tif_to_png(cropped_image, png)

In [ ]:
def calculate_max_diagonal_radius(gdf):
    diagonals = []
    for geom in gdf.geometry:
        if not geom.is_empty:
            minx, miny, maxx, maxy = geom.bounds
            diagonal = np.sqrt((maxx - minx) ** 2 + (maxy - miny) ** 2)
            diagonals.append(diagonal)
        else:
            diagonals.append(0)

    max_diagonal = max(diagonals)
    radius = max_diagonal / 2
    return diagonals, radius

def get_radius(gdf):
    # Extract bounding box values (minx, miny, maxx, maxy)
    gdf["min_lon"], gdf["min_lat"], gdf["max_lon"], gdf["max_lat"] = zip(*gdf.bounds.apply(tuple, axis=1))

    # Compute diagonals as the max difference in latitude and longitude
    gdf["diagonal"] = gdf.apply(lambda row: max(row["max_lat"] - row["min_lat"], row["max_lon"] - row["min_lon"]), axis=1)

    # Compute the radius as max_diagonal / 2
    gdf["radius"] = gdf["diagonal"] / 2

    return gdf

# Load dataset

In [ ]:
# insert path
path = '/content/drive/MyDrive/Tesi/milano_shp/milan_census_21.geojson'

In [ ]:
gdf_census = gpd.read_file(path)
gdf_census["area_sqm"] = gdf_census.geometry.area
gdf_census = get_radius(gdf_census)
gdf_census = gdf_census.to_crs(epsg='4326')
gdf_census["representative_point"] = gdf_census.geometry.representative_point()
#gdf_census = gdf_census.to_crs(epsg='3857')
gdf_census['SEZ21_ID'].nunique()

7383

In [ ]:
gdf_census.shape

(7383, 39)

In [ ]:
gdf_census[gdf_census['SEZ21_ID']==151460000222].explore()

### Select a subset of the categories or sample data

In [ ]:
# categories = [1,2,9,15,10,36,16]
# gdf_census = gdf_census[milan_census_21['COD_TIPO_S'].isin(categories)]
#census_sample = gdf_census.sample(1000, random_state=123) # Changed 'random_sate' to 'random_state'
#census_sample['COD_TIPO_S'].value_counts()

# Generate Satellite Image Given a census ID

In [ ]:
# gdf_test = gdf_census[gdf_census['COD_TIPO_S']==1].sort_values(by='SEZ21_ID', ascending=False).head(100)
# for index, row in gdf_test.iterrows():
#   census_satellite_image(gdf_census, row['SEZ21_ID'], '/content/drive/MyDrive/Tesi/satellite_images/1_test')

In [ ]:
output_path = '/content/drive/MyDrive/Tesi/satellite_images'
#census_satellite_image(gdf_census, 151920000020, output_path)

In [ ]:
df = gdf_census[gdf_census['COD_TIPO_S']==5]
for id in df['SEZ21_ID']:
  census_satellite_image(df, id, output_path)

Output streaming troncato alle ultime 5000 righe.
Downloaded image 3/35
Downloaded image 4/35
Downloaded image 5/35
Downloaded image 6/35
Downloaded image 7/35
Downloaded image 8/35
Downloaded image 9/35
Downloaded image 10/35
Downloaded image 11/35
Downloaded image 12/35
Downloaded image 13/35
Downloaded image 14/35
Downloaded image 15/35
Downloaded image 16/35
Downloaded image 17/35
Downloaded image 18/35
Downloaded image 19/35
Downloaded image 20/35
Downloaded image 21/35
Downloaded image 22/35
Downloaded image 23/35
Downloaded image 24/35
Downloaded image 25/35
Downloaded image 26/35
Downloaded image 27/35
Downloaded image 28/35
Downloaded image 29/35
Downloaded image 30/35
Downloaded image 31/35
Downloaded image 32/35
Downloaded image 33/35
Downloaded image 34/35
Downloaded image 35/35
Saving GeoTIFF. Please wait...
Image saved to 151460002988.tif
Downloaded image 1/70
Downloaded image 2/70
Downloaded image 3/70
Downloaded image 4/70
Downloaded image 5/70
Downloaded image 6/70
Dow